# Converting to Fantasy Points

In [1]:
import pandas as pd

In [2]:
player_proj = pd.read_csv('../nba-stats-csv/player_proj_df.csv')
player_info = pd.read_csv('../nba-stats-csv/player_info_df.csv')

In [3]:
proj_columns = [
    'proj_pts',
    'proj_min',
    'proj_fgm',
    'proj_fga',
    'proj_fg3m',
    'proj_fg3a',
    'proj_ftm',
    'proj_fta',
    'proj_oreb',
    'proj_dreb',
    'proj_ast',
    'proj_stl',
    'proj_tov',
    'proj_blk'                     
]

In [7]:
df_proj_stats = player_proj.loc[:, proj_columns]

In [10]:
df_proj_stats = pd.concat([player_info, df_proj_stats], axis=1)

In [12]:
proj_dictionary = {  'proj_season_id' : 'season_id',
                'proj_pts' : 'pts',
                'proj_min' : 'min',
                'proj_fgm' : 'fgm',
                'proj_fga' : 'fga',
                'proj_fg3m' : 'fg3m',
                'proj_fg3a' : 'fg3a',
                'proj_ftm' : 'ftm',
                'proj_fta' : 'fta',
                'proj_oreb' : 'oreb',
                'proj_dreb' : 'dreb',
                'proj_ast' : 'ast',
                'proj_stl' : 'stl',
                'proj_tov' : 'tov',
                'proj_blk' : 'blk'
            }

In [14]:
df_proj_stats.rename(columns = proj_dictionary, inplace=True)

In [15]:
df_proj_stats.sample(5)

,player_name,player_id,season_id,pts,min,fgm,fga,fg3m,fg3a,ftm,fta,oreb,dreb,ast,stl,tov,blk
73,Deyonta Davis,1627738,2018-19,4.981933,16.397191,2.116785,4.509211,0.049920,0.162140,0.724071,1.120528,1.474872,2.584325,0.904700,0.400349,0.643042,0.549978
5,Alex Len,203458,2018-19,8.347696,23.277941,3.200012,6.222281,0.000000,0.025929,1.936517,2.959890,2.460331,4.411809,0.742691,0.542657,1.237325,1.294121
60,DeAndre' Bembry,1627761,2018-19,5.619347,18.363281,2.137034,5.158588,0.355174,1.238335,0.976089,1.324767,0.514607,1.761332,2.169326,0.790665,1.067055,0.196785
205,Michael Carter-Williams,203487,2018-19,10.057511,24.444913,3.614661,7.944902,0.934770,2.591961,1.856036,2.410876,0.596197,2.465078,2.614916,0.860426,1.416336,0.385760
308,Zaza Pachulia,2585,2018-19,4.708466,15.200837,1.887957,4.137357,0.000000,0.043625,0.920766,1.228401,1.132377,2.250570,0.792398,0.438271,0.799345,0.252155


In [16]:
def fantasy_pts_converter(row):
    pts = row['pts']
    fgm = row['fgm']
    fga = row['fga']
    ftm = row['ftm']
    fta = row['fta']
    oreb = row['oreb']
    dreb = row['dreb']
    ast = row['ast']
    stl = row['stl']
    tov = row['tov']
    blk = row['blk']
    fantasy_pts = pts + fgm - fga + ftm - fta + oreb + dreb + ast + stl - tov + blk
    return fantasy_pts

In [18]:
fantasy_value = df_proj_stats.apply(fantasy_pts_converter, axis = 1)

In [19]:
df_proj_stats['fantasy_pts'] = fantasy_value

In [23]:
df_proj_stats.sort_values('fantasy_pts', ascending=False)

,player_name,player_id,season_id,pts,min,fgm,fga,fg3m,fg3a,ftm,fta,oreb,dreb,ast,stl,tov,blk,fantasy_pts
185,LeBron James,2544,2018-19,26.963298,37.526451,9.926719,18.992875,1.510635,4.283107,5.583248,7.591139,1.109799,6.445393,6.978517,1.493536,3.503231,0.759320,29.172586
253,Russell Westbrook,201566,2018-19,26.462878,36.794677,9.464468,19.396664,1.611749,4.605346,5.934972,7.688541,1.135222,6.152024,8.091317,1.604841,3.836465,0.501244,28.425297
101,Giannis Antetokounmpo,203507,2018-19,25.855110,36.699339,9.629458,19.317893,0.654471,1.998258,5.953134,7.784755,2.166553,7.670942,4.156653,1.396763,2.859267,1.491749,28.358448
14,Anthony Davis,203076,2018-19,24.993212,35.263440,9.414461,18.017345,0.441750,1.401594,5.723050,7.449766,2.293854,7.810872,2.281250,1.152753,2.173989,1.946802,27.975154
129,James Harden,201935,2018-19,27.432753,37.275570,8.739531,19.326112,2.441601,6.920870,7.523822,8.988620,0.891902,5.167232,6.767129,1.562489,3.839136,0.545123,26.476112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Demetrius Jackson,1627743,2018-19,3.225949,11.091111,1.075341,2.619767,0.229921,0.700412,0.816061,1.139778,0.229000,0.798923,1.226060,0.398855,0.592786,0.105853,3.523713
188,Malachi Richardson,1627781,2018-19,3.500000,11.200000,1.400000,3.800000,0.600000,2.000000,0.000000,0.100000,0.100000,1.100000,0.600000,0.500000,0.300000,0.200000,3.200000
190,Malik Beasley,1627736,2018-19,3.301039,11.540422,1.275943,3.156478,0.384840,1.183534,0.413730,0.531378,0.277839,0.797142,0.705454,0.382547,0.497197,0.090859,3.059499
124,Jake Layman,1627774,2018-19,2.692225,9.962606,0.999658,2.452452,0.258409,0.901157,0.429845,0.558704,0.225457,0.703607,0.745066,0.342058,0.404376,0.106394,2.828777


In [24]:
df_for_export = round(df_proj_stats.sort_values('fantasy_pts', ascending=False),2)

In [25]:
df_for_export.to_csv('../nba-stats-csv/projected_fantasy_pts_final_2018_19.csv', index=False)

In [26]:
df_for_export.to_csv('../nba-stats-csv/projected_fantasy_pts_final_2018_19_flan02.csv', index=False)

# Value Based Drafting

In [27]:
# The Principle:
# The value of a player is determined not by the number of points he scored, but by how much he outscores his peers at his particular position.

| Your Team | Opponent | Points Advantage | Cumulative Total |
| --------- | -------- | ---------------- | ---------------- |
|PG scores 23 pts| PG scores 20 pts| +3 for you | +3 |
|SG scores 7 pts| SG scores 13 pts| -6 for you | -3 |
|SF scores 20 pts| SF scores 5 pts| +15 for you | +12 |
|PF scores 18 pts| PF scores 19 pts| -1 for you | +11 |
|C scores 30 pts| C scores 34 pts| -4 for you | +7 |

##### Total Pts = 98 (Your Team); Total Pts = 91 (Opponent)

# Getting Our Baseline Numbers

In [28]:
import pandas as pd
import numpy as np

In [29]:
draft_df = pd.read_csv('../nba-stats-csv/Fantasy Projections - 2018_19 - 2018_espn_draft.csv', header=0)

In [30]:
draft_df.head(5)

,pick,player,position
0,1,Russell Westbrook,PG
1,2,James Harden,SG
2,3,Kevin Durant,SF
3,4,Giannis Antetokounmpo,SF
4,5,Anthony Davis,PF


In [31]:
df_top_80 = draft_df[draft_df['pick'] <= 80]

In [35]:
df_top_80.tail()

,pick,player,position
75,76,Gary Harris,SG
76,77,Lonzo Ball,PG
77,78,Rodney Hood,SG
78,79,Zach Randolph,PF
79,80,Brandon Ingram,PF


In [38]:
df_top_80.groupby('position').count() # How many players of each position do we have in this draft?

,pick,player
position,,
C,15,15
PF,19,19
PG,21,21
SF,12,12
SG,13,13


In [39]:
position_dictionary = {
    'PG': 21,
    'SG': 13,
    'SF': 12,
    'PF': 19,
    'C': 15
}

In [43]:
df1 = pd.read_csv('../nba-stats-csv/Fantasy Projections - 2018_19 - 2018_espn_player_positions.csv', header=0)
df2 = pd.read_csv('../nba-stats-csv/projected_fantasy_pts_final_2018_19.csv', header=0)

In [44]:
df1.sample()

,player_name,player_id,season_id,espn_position
303,Meyers Leonard,203086,2018-19,C


In [45]:
df2.sample()

,player_name,player_id,season_id,pts,min,fgm,fga,fg3m,fg3a,ftm,fta,oreb,dreb,ast,stl,tov,blk,fantasy_pts
272,Tim Frazier,204025,2018-19,3.91,15.76,1.47,3.65,0.3,1.09,0.71,0.93,0.34,1.3,2.69,0.92,1.03,0.09,5.82


In [76]:
df3 = df1.merge(df2, how='left', on='player_id')

In [77]:
df_final = df3[['player_name_x', 'player_id', 'season_id_x', 'espn_position', 'fantasy_pts']]

In [78]:
df_final = df_final.rename(columns={
    'player_name_x': 'player_name',
    'season_id_x': 'season_id'
})

In [79]:
df_final.to_csv('../nba-stats-csv/projected_fantasy_pts_with_positions_2018_19_formatted.csv', index=False)

In [54]:
positions = [
    'PG',
    'SG',
    'SF',
    'PF',
    'C'
]

In [63]:
def calc_baseline(pos, df):
    df_filter = df[df['espn_position'] == pos]
    x = position_dictionary.get(pos)
    replacement_player = df_filter.iloc[x]
    value_to_substract = replacement_player['fantasy_pts']
    return value_to_substract

In [64]:
baseline_points = {}
for pos in positions:
    y = calc_baseline(pos, df_final)
    baseline_points[pos] = y

In [65]:
baseline_points

{'PG': np.float64(12.09),
 'SG': np.float64(13.14),
 'SF': np.float64(13.86),
 'PF': np.float64(12.81),
 'C': np.float64(14.02)}

# Draft Preparation

In [80]:
# We can work with model importing our .csv file to spreadsheet or excel and manipulate it